<center><img src="https://i.imgur.com/zRrFdsf.png" width="700"></center> 

_____

<a id='home'></a>

# Reshaping

<a target="_blank" href="https://colab.research.google.com/github/CienciaDeDatosEspacial/code_and_data/blob/main/Reshaping.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>



Data frames have have different shapes. Let me use some columns from the data on fragility:

In [2]:
import os, pandas as pd

FragilityAll=pd.read_csv(os.path.join("archivoo","FragilityCia_isos.csv"))

# seeing long shape
fragileLong=FragilityAll.iloc[:,:3]
fragileLong

,Country,Officialstatename,InternetccTLD
0,AFGHANISTAN,The Islamic Republic of Afghanistan,.af
1,ALBANIA,The Republic of Albania,.al
2,ALGERIA,The People's Democratic Republic of Algeria,.dz
3,ANGOLA,The Republic of Angola,.ao
4,ANTIGUA AND BARBUDA,Antigua and Barbuda,.ag
...,...,...,...
166,UZBEKISTAN,The Republic of Uzbekistan,.uz
167,VIET NAM,The Socialist Republic of Viet Nam,.vn
168,YEMEN,The Republic of Yemen,.ye
169,ZAMBIA,The Republic of Zambia,.zm


You notice a DF is in long shape when you see the unit of analysis repeated in more than one cell accross the rows.

Let me turn our **long** into **wide** shape:

In [3]:
# to wide
fragileWide=pd.pivot_table(fragileLong,
               values='Total', # values to use
               index=['Country'], # unit of analysis
               columns=['Year']) # the values for NEW column
# see wide
fragileWide.head()

KeyError: 'Total'

The **wide** shape from a **pivot_table** function looks great, but pay attention:

In [3]:
fragileWide.columns

Index([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021], dtype='int64', name='Year')

As you see above, the country is the row _index_ (row name), not a column. You may want this:

In [4]:
# resetting index, keeping last index as a column
fragileWide= fragileWide.reset_index(drop=False).\
             rename_axis(index=None, columns=None) # no name for indexes

# result:
fragileWide.head()

,Country,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AFGHANISTAN,106.7,106.5,107.9,107.9,107.3,106.620768,105.0,102.901187,102.1
1,ALBANIA,65.2,63.6,61.9,61.2,60.5,60.079308,58.9,58.753811,59.0
2,ALGERIA,78.7,78.8,79.6,78.3,76.8,75.785052,75.4,74.575183,73.6
3,ANGOLA,87.1,87.4,87.9,90.5,91.1,89.440296,87.8,87.320039,89.0
4,ANTIGUA AND BARBUDA,58.0,59.0,57.8,56.2,54.8,55.611041,54.4,52.062352,54.9


Long shape is efficient in the presence of missing values:

In [5]:
# missing values in long format
fragileLong[fragileLong.isna().any(axis=1)]

,Country,Year,Total


Wide shape will create missing values when needed:

In [6]:
# what cells have missing values?
fragileWide[fragileWide.isna().any(axis=1)]

,Country,2013,2014,2015,2016,2017,2018,2019,2020,2021
27,CABO VERDE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,64.2
31,CAPE VERDE,73.7,74.1,73.5,71.5,70.1,68.000000,66.6,64.778171,NaN
45,CZECH REPUBLIC,39.9,39.4,37.4,40.8,40.1,39.047601,NaN,35.741616,NaN
46,CZECHIA,NaN,NaN,NaN,NaN,NaN,NaN,37.6,NaN,39.3
56,ESWATINI,NaN,NaN,NaN,NaN,NaN,NaN,85.3,NaN,82.5
81,ISRAEL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.0
82,ISRAEL AND WEST BANK,80.8,79.5,79.4,79.7,78.9,78.533740,76.5,75.123972,NaN
90,KYRGYZ REPUBLIC,85.7,83.9,82.2,81.1,80.3,78.634122,76.2,73.929364,NaN
91,KYRGYZSTAN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,76.4
100,MACEDONIA,68.0,66.4,64.5,67.0,66.1,64.797158,64.6,62.130864,NaN


This last request shows us problems.
As you can see, even though the data DFs were prepared by the same organization, the DFs have country names that differ among them. Here we need some **manual** changes: 

In [7]:
# prepare changes as dict:
changes={"CABO VERDE": "CAPE VERDE",
"CZECHIA":"CZECH REPUBLIC",
"SWAZILAND":"ESWATINI",
"ISRAEL AND WEST BANK":"ISRAEL",
"KYRGYZSTAN":"KYRGYZ REPUBLIC",
"NORTH MACEDONIA":"MACEDONIA",
"SLOVAKIA": "SLOVAK REPUBLIC"}

Notice I am making the changes in the long shape of the DF, so that the wide shape will work fine:

In [8]:
# make changes using 'replace':
FragilityAll.Country.replace(to_replace=changes,inplace=True)
#then

fragileLong=FragilityAll.iloc[:,:3]

Let's redo the wide reshape:

In [9]:
# to wide shape again
fragileWide=pd.pivot_table(fragileLong,
               values='Total',
               index=['Country'],
               columns=['Year']).\
            reset_index(drop=False).\
            rename_axis(index=None, columns=None)

# verify missing
fragileWide[fragileWide.isna().any(axis=1)]

,Country,2013,2014,2015,2016,2017,2018,2019,2020,2021
124,PALESTINE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,86.0


You have to be very careful when working with countries, specially when you are including or excluding countries; which may cause you hurting someone else's feelings. 

For instance, here I am just keeping **rows** with no missing values:

In [10]:
fragileWide.dropna(inplace=True) 

After dropping rows, I recommend you reset the indexes:

In [11]:
fragileWide.reset_index(drop=True,inplace=True) 

For sure, we can turn this wide shape into a long one, using the function **melt**:

In [12]:
pd.melt(fragileWide, id_vars=['Country'])

,Country,variable,value
0,AFGHANISTAN,2013,106.7
1,ALBANIA,2013,65.2
2,ALGERIA,2013,78.7
3,ANGOLA,2013,87.1
4,ANTIGUA AND BARBUDA,2013,58.0
...,...,...,...
1597,VENEZUELA,2021,92.6
1598,VIETNAM,2021,63.3
1599,YEMEN,2021,111.7
1600,ZAMBIA,2021,84.9


This would be more explicit:

In [13]:
fragileLong=pd.melt(fragileWide, #DF
        id_vars=['Country'], #key
        value_vars=list(range(2013,2022)), # columns in wide
        var_name='Year', # new name for long column
        value_name='Total')# new name for values

# verifying
fragileLong.head()

,Country,Year,Total
0,AFGHANISTAN,2013,106.7
1,ALBANIA,2013,65.2
2,ALGERIA,2013,78.7
3,ANGOLA,2013,87.1
4,ANTIGUA AND BARBUDA,2013,58.0
